# Pandas - Data Analysis Library for Python - Part 1 Basics

Original by Wes McKinney 
        
Modified by Clayton Miller (miller.clayton@arch.ethz.ch)

`Pandas` is one of the most important libraries available for data analysts. It is extremely valuable when processing time-series output data

In [ ]:
%matplotlib inline

In [ ]:
from IPython.core.display import Image
Image(filename='pandaslogo.jpg')

## Pandas video tutorial
Wes McKinney created the Pandas library and this notebook was used in a live tutorial which can be found on youtube:

In [ ]:
from IPython.display import YouTubeVideo
YouTubeVideo("w26x-z-BdWQ")

In [ ]:
from IPython.core.display import Image
Image(filename='pandasbook.jpg')

The first step in any analysis is to load the necessary libraries -- in this case `pandas` and `numpy`

In [ ]:
from pandas import *
import pandas
import numpy as np



# plt.rc('figure', figsize=(10, 6))
# pandas.set_option('notebook_repr_html',False)

Pandas Series Object: 1 dimensional data container
======

This object is a data container for vectors -- incorporating an index and string search functions

In [ ]:
s = Series(np.random.randn(5))
s

In [ ]:
labels = ['a', 'b', 'c', 'd', 'e']
s = Series(np.random.randn(5), index = labels)
s

In [ ]:
'b' in s

In [ ]:
s['b']

In [ ]:
s

In [ ]:
mapping = s.to_dict()
mapping

In [ ]:
s = Series(mapping)
s

In [ ]:
s[:3]

In [ ]:
s.index

DataFrame: 2D collection of Series
==================================

In [ ]:
df = DataFrame({'a': np.random.randn(6),
                'b': ['foo', 'bar'] * 3,
                'c': np.random.randn(6)})
df.info()

In [ ]:
df.index

In [ ]:
df

In [ ]:
df = DataFrame({'a': np.random.randn(6),
                'b': ['foo', 'bar'] * 3,
                'c': np.random.randn(6)},
               index = date_range('1/1/2000', periods=6))
df

In [ ]:
df = DataFrame({'a': np.random.randn(6),
                'b': ['foo', 'bar'] * 3,
                'c': np.random.randn(6)},
               columns=['a', 'b', 'c', 'd'])
df

Creation from nested dicts
--------------------------

These arise naturally in Python code

In [ ]:
data = {}
for col in ['foo', 'bar', 'baz']:
    for row in ['a', 'b', 'c', 'd']:
        data.setdefault(col, {})[row] = np.random.randn()
data

In [ ]:
DataFrame(data)

Data alignment
==============

In [ ]:
close_px = read_csv('stock_data.csv', index_col=0, parse_dates=True)

In [ ]:
close_px

In [ ]:
s1 = close_px['AAPL'][-20:]
s2 = close_px['AAPL'][-25:-10]
s1

In [ ]:
s2

In [ ]:
s1 + s2

In [ ]:
df = close_px.iloc[-10:, :3]
df

In [ ]:
b, c  = s1.align(s2, join='inner')
b

In [ ]:
c

In [ ]:
b, c  = s1.align(s2, join='outer')
b

In [ ]:
b, c  = s1.align(s2, join='right')


In [ ]:
df = close_px.ix[-10:, ['AAPL', 'IBM', 'MSFT']]
df

In [ ]:
df2 = df.ix[::2, ['IBM', 'MSFT']]
df2

In [ ]:
df + df2

In [ ]:
b, c = df.align(df2, join='inner')

## Truncation - clipping a datetime indexed object

In [ ]:
df

In [ ]:
df.truncate(before='2011-10-05')

In [ ]:
df.truncate(before='2011-10-05',after='2011-10-12')

## Resampling - Useful time series aggregation

In [ ]:
df

In [ ]:
df.resample('M').mean()

In [ ]:
df.resample('5D').mean()

In [ ]:
df.resample('5D').max()

## Missing Data - Filling in the Gaps

In [ ]:
dfgaps = df.resample('D').mean()
dfgaps

In [ ]:
dfgaps.dropna()

In [ ]:
dfgaps.fillna(method = 'bfill')

Transposing: no copy if all columns are same type
-------------------------------------------------

In [ ]:
df[:5].T

Columns can be any type
-----------------------

In [ ]:
n = 10
foo = DataFrame(index=range(n))
foo['floats'] = np.random.randn(n)
foo['ints'] = np.arange(n)
foo['strings'] = ['foo', 'bar'] * (n / 2)
foo['bools'] = foo['floats'] > 0
foo['objects'] = date_range('1/1/2000', periods=n)
foo

In [ ]:
foo.dtypes

N.B. transposing is not roundtrippable in this case (column-oriented data structure)

In [ ]:
foo.T.T

In [ ]:
foo.T.T.dtypes

## Function application

You can apply arbitrary functions to the rows or columns of a DataFrame

In [ ]:
df.apply(np.mean)

In [ ]:
df.apply(np.mean, axis=1)

You can get as fancy as you want

In [ ]:
close_px

In [ ]:
df.apply(lambda x: x.max() - x.min()) # np.ptp

In [ ]:
np.log(close_px)

## Plotting

Some basic plotting integration with matplotlib in Series / DataFrame

In [ ]:
close_px[['AAPL', 'IBM', 'MSFT', 'XOM']].plot();

Hierarchical indexing
---------------------

In [ ]:
index = MultiIndex(levels=[['foo', 'bar', 'baz', 'qux'],
                                   ['one', 'two', 'three']],
                           labels=[[0, 0, 0, 1, 1, 2, 2, 3, 3, 3],
                                   [0, 1, 2, 0, 1, 1, 2, 0, 1, 2]])
hdf = DataFrame(np.random.randn(10, 3), index=index,
                columns=['A', 'B', 'C'])
hdf

In [ ]:
hdf.loc['foo']

In [ ]:
hdf.loc['foo'] = 0
hdf

In [ ]:
hdf.loc['foo', 'three']

Stacking and unstacking
-----------------------

In [ ]:
tuples = zip(*[['bar', 'bar', 'baz', 'baz',
                'foo', 'foo', 'qux', 'qux'],
               ['one', 'two', 'one', 'two',
                'one', 'two', 'one', 'two']])
index = MultiIndex.from_tuples(tuples)
columns = MultiIndex.from_tuples([('A', 'cat'), ('B', 'dog'),
                                  ('B', 'cat'), ('A', 'dog')])
df = DataFrame(np.random.randn(8, 4), index=index, columns=columns)
df

In [ ]:
df2 = df.iloc[[0, 1, 2, 4, 5, 7]]
df2

In [ ]:
df.unstack()['B']

## GroupBy


In [ ]:
df = DataFrame({'A' : ['foo', 'bar', 'foo', 'bar',
                       'foo', 'bar', 'foo', 'foo'],
                'B' : ['one', 'one', 'two', 'three',
                       'two', 'two', 'one', 'three'],
                'C' : np.random.randn(8),
                'D' : np.random.randn(8)})
df

In [ ]:
for key, group in df.groupby('A'):
    print key
    print group

In [ ]:
df.groupby('A')['C'].describe().T

In [ ]:
df.groupby('A').mean()

In [ ]:
for key, group in df.groupby('A'):
    print key
    print group

In [ ]:
df.groupby(['A', 'B']).mean()

In [ ]:
df.groupby(['A', 'B'], as_index=False).mean()

GroupBy with hierarchical indexing
----------------------------------

In [ ]:
tuples = zip(*[['bar', 'bar', 'baz', 'baz',
                'foo', 'foo', 'qux', 'qux'],
               ['one', 'two', 'one', 'two',
                'one', 'two', 'one', 'two']])
index = MultiIndex.from_tuples(tuples)
columns = MultiIndex.from_tuples([('A', 'cat'), ('B', 'dog'),
                                  ('B', 'cat'), ('A', 'dog')])
df = DataFrame(np.random.randn(8, 4), index=index, columns=columns)
df

In [ ]:
df.groupby(level=0, axis=0).mean()

In [ ]:
df.stack()

In [ ]:
df.stack().mean(1).unstack()

In [ ]:
# could also have done
df.groupby(level=1, axis=1).mean()